<h4 style="text-align:right">The Course of Computational Chemisty; Peking University</h4>
***
<p><br>
<h1 style="text-align:center"><font face="宋体" size=6>5th Homework <br> Tutorial of Quantum Chemistry Programing - ver.01 </font></h1>

## 0. Members   
Qilin He, Zhi Zi, Haoming Liu, Ta Teng，Zuoran Qiao, Xin He(Leader)   
##### An old version refers https://github.com/Utenaq/2018QC-Project-Ab-initio-wavefunction-program .  
-----

## 1. compile and usage
### compile

in the main directory or under `./src`，use `make` command; it needs g++ compiler.  

### usage

| command     | term      |      |
| ---------| -------- | -------- |
| `main -h`     | get help info    |      |
| `main -d`    | default test (eg. HeH+)     | test file locates in `../test`     |
| `main -f [file.gjf]`  | normal calculation     | a series test file are under `../test`  |

  
optional test file lies in `../test`，including **H.gjf**, **He.gjf**, **H2.gjf**, **HeH.gjf**, **H4.gjf**, **CH4.gjf**.  
use as `main -f ../test/H2.gjf`.


## 2. class and type
refer to [class and type]()

## 3. handling of molecule integral  

#### Radial gaussian  and cartesian gaussian
radial gaussian gives as:  
$$ g_{lmn}(\alpha, \mathbf{r}) = \Big(\sqrt{\frac{2}{\pi}} \frac{(4\alpha)^{n+1/2}}{(2n-1)!!}\Big) r^{n-1}e^{-\alpha r^2} Y_{lm}(\theta, \varphi) $$  
but here we more often use the cartesian gaussian as:  
$$ g(A,\alpha, l,m,n) = N x_A^l y_A^m z_A^n e^{-\alpha r_A^2} $$
where by formula  
$$\int_{-\infty}^{\infty}x^{2n}e^{-\alpha x^2}=\sqrt{\frac{\pi}{\alpha}}\frac{(2n-1)!!}{(2\alpha)^n}$$
we have
$$ N =\big[\big(\frac{2\alpha}{\pi}\big)^{3/2}\frac{(4\alpha)^{l+m+n}}{(2l-1)!!(2m-1)!!(2n-1)!!} \big]^{1/2}$$

#### GTO product theorem  
$$ e^{-\alpha_1 r_A^2}e^{-\alpha_2 r_B^2} = \exp\big[-\frac{\alpha_1\alpha_2}{\alpha_1+\alpha_2}\big]\exp\big[-(\alpha_1+\alpha_2)r_P^2 \big] $$
where 
$$ \mathbf{P} \overset{def}{===} \frac{\alpha_1 \mathbf{A} + \alpha_2 \mathbf{B}}{\alpha_1+\alpha_2}$$
so let $x_A^{l_1}x_B^{l_2}$ expand at $x_P$ as: 
$$ x_A^{l_1}x_B^{l_2} = (x_P - \overline{PA}_x)_A^{l_1} (x_P - \overline{PB}_x)_B^{l_2} \\
= \sum_{i_1=0}^{l_1} \sum_{i_2=0}^{l_2} (-1)^{i_1+i_2} C_{l_1}^{i_1}C_{l_2}^{i_2} (\overline{PA}_x)^{l_1-i_1} (\overline{PB}_x)^{l_2-i_2} x_P^{i_1+i_2} 
\\
= \sum_l^{l_1+l_2} (-1)^{l} f_l (l_1,l_2,\overline{PA}_x, \overline{PB}_x) x_P^l
$$
where 
$$ f_l(l_1,l_2,a,b) = \sum_{i_1=0}^{l_1} \sum_{i_2=0}^{l_2} \delta_{l,l_1+l_2} C_{l_1}^{i_1}C_{l_2}^{i_2} a^{l_1-i_1} b^{l_2-i_2} $$
and so on:  
$$
g(A,\alpha_1,l_1,m_1,n_1)g(B,\alpha_2,l_2,m_2,n_2) 
\\
= \sum_{l=0}^{l_1+l_2}\sum_{m=0}^{m_1+m_2}\sum_{n=0}^{n_1+n_2}
(-1)^{l+m+n} f_l (l_1,l_2,\overline{PA}_x, \overline{PB}_x)
f_m (m_1,m_2,\overline{PA}_y, \overline{PB}_y)
\\
\times
f_n (n_1,n_2,\overline{PA}_z, \overline{PB}_z)
x_P^l y_P^m z_P^n 
\exp\big[-\frac{\alpha_1\alpha_2}{\alpha_1+\alpha_2}\big]\exp\big[-(\alpha_1+\alpha_2)r_P^2 \big]
$$

#### useful integral formula  
1. 
$$ \int_{-\infty}^{\infty} x^{2n}e^{-\alpha x^2}dx = \big(\frac{\pi}{\alpha}\big)^{1/2}
\frac{(2n-1)!!}{(2\alpha)^n}
$$
2. 
$$
\int_{-\infty}^{\infty} x^{2n+1}e^{-\alpha x^2}dx = 0
$$
3. 
$$
\int_{-\infty}^{\infty} e^{ixy}x^{n}e^{-\alpha x^2}dx = i^n \big(\frac{\pi}{\alpha}\big)^{1/2}\big(\frac{1}{2\sqrt{\alpha}}\big)^n H_n\big(\frac{y}{2\sqrt{\alpha}}\big) e^{-y^2/4\alpha}
$$
where $H_n$ is Hermite polynomial:  
$$
H_n(z) = (-1)^n e^{z^2} \frac{d^n}{dz^n} e^{-z^2}
= n!\sum_{i=0}^{[n/2]}\frac{(-1)^i}{i!(n-2i)!}(2z)^{n-2i}
$$
4. 
$$
\frac{1}{r} = \frac{1}{2\pi^2} \int e^{i\mathbf{k}\cdot\mathbf{r}}\frac{1}{k^2} d\mathbf{k}
$$
5. 
$$
e^{-\sigma k^2} = 2\sigma k^2 \int_0^1 S^{-3} e^{-\sigma k^2/S^2} dS
$$
6. defines
$$
F_n (t) = \int_0^1 u^{2n} e^{-t u^2} du
$$

#### Radial integral formula  (1s type)
for n=1, l=m=0, or called 1s function, such integral is derived by **Boys** and **Shavitt**.  
1. S integral  
$$ S = \int e^{-a(\mathbf{r}-\mathbf{A})^2 - b(\mathbf{r}-\mathbf{B})^2} 
= \big( \frac{\pi}{a+b} \big)^{3/2} e^{-ab/(a+b)\cdot \overline{AB}^2 }
$$
2. T integral  
$$ T = \int e^{-a(\mathbf{r}-\mathbf{A})^2}(-)\frac{1}{2}\nabla^2 e^{- b(\mathbf{r}-\mathbf{B})^2} 
= (\frac{ab}{a+b})(3-\frac{2ab}{a+b}) e^{-ab/(a+b)\cdot \overline{AB}^2 }
$$
3. V integral  
$$ V = \int e^{-a(\mathbf{r}-\mathbf{A})^2} \frac{1}{|\mathbf{r}-\mathbf{C}|} e^{- b(\mathbf{r}-\mathbf{B})^2} 
= (\frac{2\pi}{a+b}) F_0[(a+b)\overline{CP}^2] e^{-ab/(a+b)\cdot \overline{AB}^2 }
$$  
4. ERI integral 
$$ I = \int e^{-a(\mathbf{r_1}-\mathbf{A})^2 - b(\mathbf{r_1}-\mathbf{B})^2} \frac{1}{r_{12}} e^{- c(\mathbf{r_2}-\mathbf{C})^2 - c(\mathbf{r_2}-\mathbf{D})^2} 
\\
= \frac{2\pi^{5/2}}{(a+b)(c+d)\sqrt{a+b+c+d}} F_0[\frac{(a+b)(c+d)}{a+b+c+d}\cdot\overline{PQ}^2] 
e^{-ab/(a+b)\cdot \overline{AB}^2 - cd/(c+d)\cdot \overline{CD}^2 }
$$  

#### Cartesian integral formula  
* overlap integral (S integral)  
by GTO product theorem, we can easily get:  
$$
S =\big(\frac{\pi}{a+b}\big)^{3/2} e^{-ab/(a+b)\cdot \overline{AB}^2}\tilde{S}_{l_1l_2}
\tilde{S}_{m_1m_2} \tilde{S}_{n_1n_2}
$$
where  
$$
\tilde{S}_{l_1l_2} = \sum_{i=0}^{[\frac{l_1+l_2}{2}]} f_{2i}(l_1,l_2,\overline{PA}_x,\overline{PB}_x) \frac{(2i-1)!!}{2^i (a+b)^i}
$$

* kinetic integral (T integral)  
note $g(A,a,l,m,n)$ as $\mid aAlmn \rangle$, it's easy to show that its derivatives have following properties:  
$$\frac{\partial}{\partial x_A} \mid aAlmn \rangle = (lx_A^{-1}-2ax_A) \mid aAlmn \rangle$$
$$\frac{\partial^2}{\partial x_A^2} \mid aAlmn \rangle = (l(l-1)x_A^{-2}-2a(2l+1)+4a^2x_A^2) \mid aAlmn \rangle $$  
so that:
$$
T = \alpha_2(2(l_2+m_2+n_2)+3)S_{l_1m_1n_1;l_2m_2n_2} 
\\
- 2\alpha_2^2
(S_{l_1m_1n_1;(l_2+2)m_2n_2} + S_{l_1m_1n_1;l_2(m_2+2)n_2} + S_{l_1m_1n_1;l_2m_2(n_2+2)})
\\
- \frac{1}{2}(l_2(l_2-1)S_{l_1m_1n_1;(l_2-2)m_2n_2} + m_2(m_2-1)S_{l_1m_1n_1;l_2(m_2-2)n_2} + n_2(n_2-1)S_{l_1m_1n_1;l_2m_2(n_2-2)})
$$

* nuclear Coulomb integral (V integral; without nucleus-nucleus Coulomb integral!)  
$$
V_{ABC} = \int g(A,a,l_1,m_1,n_1)\frac{1}{r_C}g(B,b,l_2,m_2,n_2)d\tau
\\
=N_AN_B \frac{1}{2\pi}e^{-ab/(a+b)\cdot \overline{AB}^2} \sum_{lmn} f_l(l_1,l_2,\overline{PA}_x, \overline{PB}_x) f_m(m_1,m_2,\overline{PA}_y, \overline{PB}_y) f_n(n_1,n_2,\overline{PA}_z, \overline{PB}_z)
\\
\times \int e^{i\mathbf{k}\cdot \mathbf{r_{cp}}} \frac{1}{k^2}d\mathbf{k} \int e^{ik_x x}x^l e^{-(a+b)x^2} dx \int e^{ik_y y}y^m e^{-(a+b)y^2} dy \int e^{ik_y y}y^n e^{-(a+b)z^2} dz
$$  
we here omit the derivation, giving result directly as:  
$$ V_{ABC} = 
\frac{2\pi}{a+b}e^{-ab/(a+b)\cdot\overline{AB}^2} \sum_{\nu}^{N}\sum_{I=0}^{l_1+l_2}
\sum_{J=0}^{m_1+m_2}\sum_{K=0}^{n_1+n_2} 
G_{l_1l_2}^{I}(A_x,B_x,C_x)
\\
\times G_{m_1m_2}^{J}(A_y,B_y,C_y) G_{n_1n_2}^{K}(A_z,B_z,C_z)
F_{\nu}[(a+b)\overline{PC}^2]\delta_{\nu,I+J+K}
$$
where
$$
G_{l_1l_2}^{I}(A_x,B_x,C_x) = \sum_{i=0}^{l_1+l_2}\sum_{r=0}^{[i/2]} \sum_{u=0}^{[i/2]-r} (-1)^i f_i (l_1,l_2,\overline{PA}_x,\overline{PB}_x)
\\ \times
\frac{(-1)^u i!(\overline{PC}_x)^{i-2r-2u}}{r!u!(i-2r-2u)!} \big(\frac{1}{4(a+b)}\big)^{r+u} \delta_{I,i-2r-u}
$$

* ERI integral  
$$
I = \frac{2\pi^2}{(a+b)(c+d)}\big(\frac{1}{a+b+c+d}\big)^{1/2} e^{-ac/(a+b)\cdot \overline{AB}^2 - cd/(c+d)\cdot \overline{CD}^2} 
\\ \times
\sum_{\nu=0}^{N} F_{\nu}(\overline{PQ}^2/(4\gamma))\sum_{I=0}^{l_1+l_2+l_3+l_4}
\sum_{J=0}^{m_1+m_2+m_3+m_4}\sum_{K=0}^{n_1+n_2+n_3+n_4}
D_{l_1l_2l_3l_4}^I (A_x,B_x,C_x,D_x)
\\ \times
D_{m_1m_2m_3m_4}^J (A_y,B_y,C_y,D_y)
D_{n_1n_2n_3n_4}^K (A_z,B_z,C_z,D_z)\delta_{\nu,I+J+K}
$$
where
$$
\gamma = (a+b+c+d)/[4(a+b)(c+d)]
\\
D_{l_1l_2l_3l_4}^I (A_x,B_x,C_x,D_x) = \sum_{l=0}^{l_1+l_2}\sum_{l'=0}^{l_3+l_4}
\sum_{u=0}^{[(l+l')/2]} \frac{(-1)^u (l+l')! \overline{PQ}_x^{l+l'-2u}}{u!(l+l'-2u)!\gamma^{l+l'-u}}
\\ 
\times
H_{l_1l_2}^l (\overline{PA}_x,\overline{PB}_x,a+b) (-1)^{l'} 
H_{l_3l_4}^{l'} (\overline{QC}_x,\overline{QD}_x,c+d)\delta_{I,l+l'-u}
$$
where
$$
H_{l_1l_2}^l (\overline{PA}_x,\overline{PB}_x,\beta)
= \sum_{i=0}^{l_1+l_2}\sum_{r=0}^{[i/2]} \frac{i!}{r!(i-2r)!(4\beta)^{i-r}} f_i (l_1,l_2,\overline{PA}_x,\overline{PB}_x)\delta_{l,i-2r}
$$

## 4. SCF procedure

#### procedure  
1. read and construct basis set space.  
4. calculate S、H、ERI inetgral matrix. Symmetric orthogonalize S to get transfrom matrix X and its inverse Y.
5. assume H as Fock matrix，solve the coefficents, then get initial density matrix P for guess.
6. from P、H、G matrix to claculate Fock matrix, by transform X to F'.
7. solve the eigenvalue problem of F', get its eigenvalues e and eigenvectors C', and by transform X to C.
8. determine the population of MOs, then calculate energy E、density matrix P.  
9. if E and P is consistent with the last step, jump loop.

## 5. Symmetry(lack)

## 6. Test and result


| test     | result \[au\]      | Gauss09 \[au\]  |  remark |
| ---------| -------- | -------- |--------|
| H     | -0.42244193     | -0.4982329  | not suit for open-shell|
| He     | -2.85516043     | -2.8551604  | yes|
| H2     | -1.11003090     | -1.1100309  | yes |
| HeH+   | -2.89478689     | -2.8947869  | yes |
| H4     | -1.80246920     | -1.7251712  | no |
| CH4    | vibration      | -39.9119255   | no |